# Dotin Data Prep and EDA

In [12]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
import json

## Load Data

In [13]:
mouse = pd.read_csv("../Data/Raw/mouse_paths.csv")
votes = pd.read_csv("../Data/Raw/votes.csv")

In [14]:
mouse.head()

,id,user_id,path,created_at,updated_at
0,1,365,"{""frames"":[[""s"",0,0],[""m"",601,167,0.2200000286...",2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652
1,2,371,"{""frames"":[[""s"",0,0],[""m"",707,224,0.1179997920...",2019-03-06 10:13:30.862296,2019-03-06 10:13:30.862296
2,3,390,"{""frames"":[[""s"",0,0],[""m"",683,356,0.2530000209...",2019-03-07 11:03:09.178940,2019-03-07 11:03:09.178940
3,4,376,"{""frames"":[[""s"",0,0],[""m"",569,158,0.4359998703...",2019-03-07 11:05:58.261066,2019-03-07 11:05:58.261066
4,5,396,"{""frames"":[[""s"",0,0],[""m"",456,176,0.1780002117...",2019-03-07 11:07:43.482881,2019-03-07 11:07:43.482881


In [15]:
# ignore this cell
# dict(json.loads(mouse.path[0]))
# dict(dict(json.loads(mouse.path[0]))['window'])["width"]
# json.loads(mouse.path[0])
# mouse.path[409]

In [16]:
# these indices contain incomplete frames data
broken = [393,409,416,428,582,613,687,809]
broken_uids = [mouse.user_id[i] for i in broken]; broken_uids 

[1236, 751, 1292, 1312, 1555, 1579, 1703, 1840]

In [17]:
mouse.drop(broken,axis=0,inplace=True) # drop the broken indicies 

In [18]:
# preparing the mouse data for flattening vy first adding the time elapsed, window_x and window_y cols
time_elapsed, window_x, window_y = [],[],[]
    
for index,row in mouse.iterrows():
    d = dict(json.loads(mouse.path[index]))
    time_elapsed.append(d["timeElapsed"])
    window_x.append(dict(d['window'])["width"])
    window_y.append(dict(d['window'])["height"])
    
mouse["timeElapsed"] = time_elapsed
mouse["window_x"] = window_x
mouse["window_y"] = window_y

In [19]:
# function to create a new row for each frames movement 
def flatten_df(df):
    """
    input mouse df with a json column, path, and return df with flattened df cols
    """
    user_id, action, cord_x, cord_y, radio, time_since = [],[],[],[],[],[]
    for index,row in df.iterrows():
        f = dict(json.loads(row.path))['frames']
        for i in range(len(f)):
            user_id.append(row.user_id)
            action.append(f[i][0])
            cord_x.append(f[i][1])
            cord_y.append(f[i][2])
            if f[i][0] == "c":
                assert len(f[i]) == 5
                radio.append(f[i][3])
                time_since.append(f[i][4])
            else:
                radio.append(np.nan)
                time_since.append(f[i][3]) if i != 0 else time_since.append(0) # since len(f[i][0]) = 3 i.e. [s,0,0], indexing at 3 results in an error
                
    res = pd.DataFrame({"user_id":user_id,
                      "action":action,
                      "cord_x":cord_x,
                      "cord_y":cord_y,
                      "radio":radio,
                      "time_since":time_since})
    res.time_since.replace("None",np.nan,inplace=True) # replace None string with np.nan for time_since
    df.drop("path",inplace=True,axis=1)
    return pd.merge(res, df, how="left", on="user_id")

In [20]:
mouse_flat = flatten_df(mouse)

In [21]:
mouse_flat.shape

(8123046, 12)

In [22]:
mouse_flat.to_csv("../Data/mouse-flat.csv")
# mouse_flat = pd.read_csv(path/"mouse-flat.csv")

In [23]:
mouse_flat.head()

,user_id,action,cord_x,cord_y,radio,time_since,id,created_at,updated_at,timeElapsed,window_x,window_y
0,365,s,0,0.0,NaN,0.000,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860
1,365,m,601,167.0,NaN,0.220,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860
2,365,m,602,166.0,NaN,0.304,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860
3,365,c,602,166.0,None,0.432,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860
4,365,m,602,166.0,NaN,0.518,1,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860


# EDA

## Mouse Data

In [24]:
mouse.shape 

(802, 7)

In [25]:
mouse.head()

,id,user_id,created_at,updated_at,timeElapsed,window_x,window_y
0,1,365,2019-03-05 06:56:35.848652,2019-03-05 06:56:35.848652,245.925,1600,860
1,2,371,2019-03-06 10:13:30.862296,2019-03-06 10:13:30.862296,192.108,1853,1053
2,3,390,2019-03-07 11:03:09.178940,2019-03-07 11:03:09.178940,117.694,1600,860
3,4,376,2019-03-07 11:05:58.261066,2019-03-07 11:05:58.261066,156.110,1280,680
4,5,396,2019-03-07 11:07:43.482881,2019-03-07 11:07:43.482881,207.118,1366,728


In [26]:
# the created_at and updated_at values are exactly the same
sum((mouse.created_at != mouse.updated_at))

0

In [27]:
# how many users data do we have (non broken)
len(mouse.user_id.unique())

730

There are 802 instances so some users are repeated

In [28]:
# which users are repeated and how many times
mouse.user_id.value_counts()[mouse.user_id.value_counts() > 1]

642     5
990     5
1077    4
386     4
647     4
685     3
989     3
1017    3
796     3
971     3
404     3
396     3
646     2
422     2
1022    2
764     2
645     2
654     2
659     2
1114    2
932     2
444     2
667     2
652     2
641     2
1441    2
673     2
877     2
869     2
866     2
401     2
630     2
395     2
839     2
828     2
1170    2
383     2
794     2
1174    2
798     2
749     2
803     2
691     2
962     2
683     2
999     2
718     2
704     2
679     2
727     2
1011    2
1722    2
1537    2
Name: user_id, dtype: int64

In [29]:
# descriptive stats on elapsed time
mouse.timeElapsed.describe()

count     802.000000
mean      487.030003
std       309.765446
min        10.930000
25%       252.644250
50%       426.907500
75%       649.207500
max      2856.612000
Name: timeElapsed, dtype: float64

In [30]:
# descriptitve stats on window_x
mouse.window_x.describe()

count     802.000000
mean     1346.420200
std       316.259552
min         0.000000
25%      1360.000000
50%      1366.000000
75%      1366.000000
max      2576.000000
Name: window_x, dtype: float64

In [31]:
# descriptitve stats on window_y
mouse.window_y.describe()

count     802.000000
mean      772.829177
std       145.474554
min         0.000000
25%       728.000000
50%       728.000000
75%       819.250000
max      1781.000000
Name: window_y, dtype: float64

In [32]:
# describe the stats on number of frames per user_id-id instance.
mouse_flat.groupby(["user_id","id"]).count()["action"].describe()

count      802.000000
mean     10128.486284
std       6664.522545
min        108.000000
25%       6163.750000
50%       9385.000000
75%      12899.500000
max      48065.000000
Name: action, dtype: float64

The average # of frames for any one instance is ~10.1k 

In [33]:
# let's see the average aspect ratio
print("16:9",16/9)
print("4:3",4/3)
mouse["aspect_ratio"] = mouse["window_x"] / mouse["window_y"]
mouse.aspect_ratio.describe()

16:9 1.7777777777777777
4:3 1.3333333333333333


count    798.000000
mean       1.744296
std        0.314581
min        0.537859
25%        1.766423
50%        1.868132
75%        1.876374
max        2.932790
Name: aspect_ratio, dtype: float64

Most windows are close to the 16:9 aspect ratio